### 1. Importar librerías

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import joblib

### 2. Cargar modelo

In [2]:
# Cargar el modelo guardado
modelo = joblib.load("..\prediccion_reforestacion\modelo\modelo_hibrido_0.9269.pkl")

# Cargar los datos originales para obtener todas las estaciones y distritos
df_original = pd.read_csv('..\prediccion_reforestacion\data\data_monitoreo_contaminantes_filtrado_best.csv')

# Datos históricos de PM10
historico_pm10 = {
    2015: 68.51, 2016: 71.85, 2017: 73.25, 2018: 69.36, 
    2019: 56.26, 2020: 53.22, 2021: 60.80, 2022: 56.51,
    2023: 41.89, 2024: 46.72
}

# Datos de arborización SERPAR
arboles_serpar = {
    2020: 82637,
    2021: 160932,
    2022: 150136,
    2023: 86957,
    2024: 115548
}

# Constantes globales
TASA_SUPERVIVENCIA = 0.7
ARBOLES_PROMEDIO = np.mean(list(arboles_serpar.values()))
VOLUMEN_AIRE = 2819000000 * 10  # área de Lima * altura promedio de mezcla

<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:5: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:5: SyntaxWarning: invalid escape sequence '\p'
C:\Users\marli\AppData\Local\Temp\ipykernel_3104\3207324733.py:2: SyntaxWarning: invalid escape sequence '\p'
  modelo = joblib.load("..\prediccion_reforestacion\modelo\modelo_hibrido_0.9269.pkl")
C:\Users\marli\AppData\Local\Temp\ipykernel_3104\3207324733.py:5: SyntaxWarning: invalid escape sequence '\p'
  df_original = pd.read_csv('..\prediccion_reforestacion\data\data_monitoreo_contaminantes_filtrado_best.csv')
c:\Users\marli\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-ma

ValueError: node array from the pickle has an incompatible dtype:
- expected: {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}
- got     : [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]

### 3. Preparar escenarios

In [16]:
def crear_datos_prediccion(start_year=2025, end_year=2035):
    """
    Crea DataFrame con todas las combinaciones necesarias para la predicción
    """
# Obtener todas las estaciones y distritos únicos
    estaciones = df_original['ESTACION'].unique()
    distritos = df_original['DISTRITO'].unique()
    
    datos = []
    for año in range(start_year, end_year + 1):
        for mes in range(1, 13):
            for estacion, distrito in zip(estaciones, distritos):
                for hora in range(24):
                    dia_semana = pd.Timestamp(year=año, month=mes, day=1).weekday()
                    es_fin_semana = 1 if dia_semana >= 5 else 0
                    datos.append({
                        'ESTACION': estacion,
                        'DISTRITO': distrito,
                        'HORA': hora,
                        'DIA_SEMANA': dia_semana,
                        'MES': mes,
                        'AÑO': año,
                        'ES_FIN_DE_SEMANA': es_fin_semana
                    })
    return pd.DataFrame(datos)

def simular_escenarios(df_pred, start_year=2025):
    """
    Simula escenarios con datos más realistas
    """
    prediccion_base = modelo.predict(df_pred)
    prediccion_con_arboles = prediccion_base.copy()
    
    # Calcular reducción por árbol (5217.4 g/año)
    reduccion_diaria_por_arbol = 5217.4 / 365
    
    años_unicos = df_pred['AÑO'].unique()
    arboles_acumulados = {}
    
    for i, año in enumerate(años_unicos):
        # Árboles acumulados considerando supervivencia
        n_arboles = int(ARBOLES_PROMEDIO * TASA_SUPERVIVENCIA * (i + 1))
        arboles_acumulados[año] = n_arboles
        
        reduccion_diaria = reduccion_diaria_por_arbol * n_arboles
        reduccion_concentracion = (reduccion_diaria * 1000) / VOLUMEN_AIRE
        
        mask_año = df_pred['AÑO'] == año
        prediccion_con_arboles[mask_año] = np.maximum(0, 
            prediccion_con_arboles[mask_año] - reduccion_concentracion)
    
    return prediccion_base, prediccion_con_arboles, arboles_acumulados

def categorizar_calidad_aire(valor):
    if valor <= 54:
        return 'Buena'
    elif valor <= 154:
        return 'Moderada'
    elif valor <= 254:
        return 'Insalubre para grupos sensibles'
    else:
        return 'Insalubre'


In [17]:
# Crear datos para predicción
df_prediccion = crear_datos_prediccion(2025, 2035)

# Realizar simulación
pred_base, pred_arboles, arboles_acumulados = simular_escenarios(df_prediccion)

# Crear visualización
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=('Evolución histórica y proyección del PM10',
                                  'Reducción proyectada por arborización'))

# Añadir datos históricos
fig.add_trace(
    go.Scatter(x=list(historico_pm10.keys()),
               y=list(historico_pm10.values()),
               name='Histórico PM10',
               line=dict(color='gray', dash='dash')),
    row=1, col=1
)

# Procesar resultados por año con margen de error
resultados = []
margen_error = 0.05  # 5% de margen de error

for año in range(2025, 2036):
    mask_año = df_prediccion['AÑO'] == año
    pm10_sin_arboles = np.mean(pred_base[mask_año])
    pm10_con_arboles = np.mean(pred_arboles[mask_año])
    
    # Calcular márgenes de error
    error_sin_arboles = pm10_sin_arboles * margen_error
    error_con_arboles = pm10_con_arboles * margen_error
    
    resultados.append({
        'Año': año,
        'PM10_Sin_Arboles': pm10_sin_arboles,
        'PM10_Sin_Arboles_Min': pm10_sin_arboles - error_sin_arboles,
        'PM10_Sin_Arboles_Max': pm10_sin_arboles + error_sin_arboles,
        'PM10_Con_Arboles': pm10_con_arboles,
        'PM10_Con_Arboles_Min': pm10_con_arboles - error_con_arboles,
        'PM10_Con_Arboles_Max': pm10_con_arboles + error_con_arboles,
        'Reduccion': pm10_sin_arboles - pm10_con_arboles,
        'Arboles_Acumulados': arboles_acumulados[año]
    })

resultados_df = pd.DataFrame(resultados)

# Añadir líneas de predicción con bandas de error
fig.add_trace(
    go.Scatter(x=resultados_df['Año'],
               y=resultados_df['PM10_Sin_Arboles'],
               name='Proyección sin árboles',
               line=dict(color='red')),
    row=1, col=1
)

# Añadir banda de error para proyección sin árboles
fig.add_trace(
    go.Scatter(
        x=resultados_df['Año'].tolist() + resultados_df['Año'].tolist()[::-1],
        y=resultados_df['PM10_Sin_Arboles_Max'].tolist() + resultados_df['PM10_Sin_Arboles_Min'].tolist()[::-1],
        fill='toself',
        fillcolor='rgba(255,0,0,0.2)',
        line=dict(color='rgba(255,0,0,0)'),
        name='Margen de error sin árboles',
        showlegend=True
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=resultados_df['Año'],
               y=resultados_df['PM10_Con_Arboles'],
               name='Proyección con árboles',
               line=dict(color='green')),
    row=1, col=1
)

# Añadir banda de error para proyección con árboles
fig.add_trace(
    go.Scatter(
        x=resultados_df['Año'].tolist() + resultados_df['Año'].tolist()[::-1],
        y=resultados_df['PM10_Con_Arboles_Max'].tolist() + resultados_df['PM10_Con_Arboles_Min'].tolist()[::-1],
        fill='toself',
        fillcolor='rgba(0,255,0,0.2)',
        line=dict(color='rgba(0,255,0,0)'),
        name='Margen de error con árboles',
        showlegend=True
    ),
    row=1, col=1
)

# Añadir barras de reducción
fig.add_trace(
    go.Bar(x=resultados_df['Año'],
           y=resultados_df['Reduccion'],
           name='Reducción PM10',
           marker_color='blue'),
    row=2, col=1
)

# Añadir líneas de referencia para categorías de calidad del aire
categorias = {
    'Buena': 54,
    'Moderada': 154
}

for categoria, valor in categorias.items():
    fig.add_hline(y=valor, line_dash="dash", line_color="gray",
                  annotation_text=categoria, row=1, col=1)

# Actualizar diseño
fig.update_layout(
    title='Simulación de PM10 en Lima Metropolitana (2015-2035)',
    height=800,
    showlegend=True,
    xaxis=dict(rangeslider=dict(visible=True))
)

fig.show()

In [18]:
# Imprimir estadísticas con margen de error
print("\nEstadísticas de la simulación:")
print(f"\nPromedio de árboles plantados por año (SERPAR): {int(ARBOLES_PROMEDIO):,}")
print(f"Tasa de supervivencia considerada: {TASA_SUPERVIVENCIA*100}%")
print(f"Margen de error considerado: ±{margen_error*100}%")

for resultado in resultados:
    print(f"\nAño {resultado['Año']}:")
    print(f"Árboles acumulados: {resultado['Arboles_Acumulados']:,}")
    print(f"Reducción promedio: {resultado['Reduccion']:.2f} µg/m³")
    print(f"PM10 sin árboles: {resultado['PM10_Sin_Arboles']:.2f} µg/m³ (±{resultado['PM10_Sin_Arboles']*margen_error:.2f})")
    print(f"PM10 con árboles: {resultado['PM10_Con_Arboles']:.2f} µg/m³ (±{resultado['PM10_Con_Arboles']*margen_error:.2f})")


Estadísticas de la simulación:

Promedio de árboles plantados por año (SERPAR): 119,242
Tasa de supervivencia considerada: 70.0%
Margen de error considerado: ±5.0%

Año 2025:
Árboles acumulados: 83,469
Reducción promedio: 0.04 µg/m³
PM10 sin árboles: 47.65 µg/m³ (±2.38)
PM10 con árboles: 47.61 µg/m³ (±2.38)

Año 2026:
Árboles acumulados: 166,938
Reducción promedio: 0.08 µg/m³
PM10 sin árboles: 47.40 µg/m³ (±2.37)
PM10 con árboles: 47.31 µg/m³ (±2.37)

Año 2027:
Árboles acumulados: 250,408
Reducción promedio: 0.13 µg/m³
PM10 sin árboles: 47.33 µg/m³ (±2.37)
PM10 con árboles: 47.21 µg/m³ (±2.36)

Año 2028:
Árboles acumulados: 333,877
Reducción promedio: 0.17 µg/m³
PM10 sin árboles: 48.26 µg/m³ (±2.41)
PM10 con árboles: 48.09 µg/m³ (±2.40)

Año 2029:
Árboles acumulados: 417,347
Reducción promedio: 0.21 µg/m³
PM10 sin árboles: 48.00 µg/m³ (±2.40)
PM10 con árboles: 47.79 µg/m³ (±2.39)

Año 2030:
Árboles acumulados: 500,816
Reducción promedio: 0.25 µg/m³
PM10 sin árboles: 47.77 µg/m³ (±2.39